In [1]:
# demo_two_subgraphs.py
from __future__ import annotations
from typing import Annotated, TypedDict
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langchain_core.messages import AIMessage, AnyMessage
from langgraph.checkpoint.memory import InMemorySaver

# ────────────────────────────────────────────────────────────
# 1. STATE
class ChatState(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]   # list-merging reducer
    current_phase: str
    phase_complete: bool


# ────────────────────────────────────────────────────────────
# 2. FIRST SUB-GRAPH  (unchanged)
def child_node(state: ChatState) -> ChatState:
    return {
        "messages": [AIMessage("👋 Hi from inside the *first* sub-graph!")],
        "current_phase": "child_phase",
        "phase_complete": True,
    }

first_builder = StateGraph(ChatState)
first_builder.add_node("child", child_node)
first_builder.set_entry_point("child")
first_builder.add_edge("child", END)
analysis_subgraph = first_builder.compile()


# ────────────────────────────────────────────────────────────
# 3. SECOND SUB-GRAPH  (new)
def extra_child(state: ChatState) -> ChatState:
    return {
        "messages": [AIMessage("🚀 Greetings from the *second* sub-graph!")],
        "current_phase": "extra_phase",
        "phase_complete": True,
    }

second_builder = StateGraph(ChatState)
second_builder.add_node("extra_child", extra_child)
second_builder.set_entry_point("extra_child")
second_builder.add_edge("extra_child", END)
extra_subgraph = second_builder.compile()


# ────────────────────────────────────────────────────────────
# 4. ROOT NODE  (plain node in parent graph)
def root_node(state: ChatState) -> ChatState:
    return {
        "messages": [AIMessage("🗣️ Hello from the *main* node!")],
        "current_phase": "root_phase",
        "phase_complete": True,
    }


# ────────────────────────────────────────────────────────────
# 5. PARENT GRAPH
parent = StateGraph(ChatState)
parent.add_node("root", root_node)                     # plain node
parent.add_node("analysis_subgraph", analysis_subgraph)
parent.add_node("extra_subgraph",    extra_subgraph)
parent.set_entry_point("root")                         # execution order
parent.add_edge("root", "analysis_subgraph")
parent.add_edge("analysis_subgraph", "extra_subgraph")
parent.add_edge("extra_subgraph", END)

graph = parent.compile(checkpointer=InMemorySaver())


# ────────────────────────────────────────────────────────────
# 6. RUN & STREAM
init_state: ChatState = {
    "messages": [],
    "current_phase": "",
    "phase_complete": False,
}
config = {"configurable": {"thread_id": "demo"}}

print("── live stream with subgraphs=True ──")
for chunk in graph.stream(
        init_state, config,
        stream_mode=["messages"],
        subgraphs=True):
        

        state   = graph.get_state(config, subgraphs=True).values

        print(f'Intermediate state - {chunk}')
        print(f'Global state: {state}')

print("\n── final merged state ──")
final_state = graph.invoke(init_state, config)
for m in final_state["messages"]:
    role = m.__class__.__name__.replace("Message", "").lower()
    print(f"{role}: {m.content}")


── live stream with subgraphs=True ──
Intermediate state - (('root:fe2d13d0-19aa-30bd-7299-f029befeeea0',), 'messages', (AIMessage(content='🗣️ Hello from the *main* node!', additional_kwargs={}, response_metadata={}, id='f307d93c-219c-41f7-9323-a167cfe652fe'), {'thread_id': 'demo', 'langgraph_step': 1, 'langgraph_node': 'root', 'langgraph_triggers': ('branch:to:root',), 'langgraph_path': ('__pregel_pull', 'root'), 'langgraph_checkpoint_ns': 'root:fe2d13d0-19aa-30bd-7299-f029befeeea0'}))
Global state: {'messages': [], 'current_phase': '', 'phase_complete': False}
Intermediate state - (('analysis_subgraph:3ed0c3ce-13ac-cced-e4e0-b5002adb2642', 'child:67dc9201-c4fb-fd03-58bc-81fd5546f77c'), 'messages', (AIMessage(content='👋 Hi from inside the *first* sub-graph!', additional_kwargs={}, response_metadata={}, id='1bf6c391-8d67-410d-8c68-500500d6d8e5'), {'thread_id': 'demo', 'langgraph_step': 1, 'langgraph_node': 'child', 'langgraph_triggers': ('branch:to:child',), 'langgraph_path': ('__prege